# Querying chpaf MoPo

In [ ]:
import rdflib
import pandas as pd

# Set pandas to display wide columns
pd.set_option('max_colwidth', 1000)

g = rdflib.Graph()
g.parse("../examples/mopo.ttl")

## Rejected MoPo

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?decision_entity a paf:DecisionEntity;
        paf:acceptance paf:Rejected.

} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

## Chapter 1

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?department ?office ?id ?title ?description ?answer WHERE {

    ?registration_activity a paf:RegistrationActivity.

    ?identifier_entity prov:wasGeneratedBy ?registration_activity;
        schema:identifier ?id.
    
    ?registration_entity prov:wasGeneratedBy ?registration_activity;
        schema:title ?title;
        schema:description ?description;
        chpaf:department ?department;
        chpaf:office ?office.

    ?proposal_creation_activity prov:wasInformedBy+ ?registration_activity.

    ?proposal_entity prov:wasGeneratedBy ?proposal_creation_activity;
        chpaf:proposal ?answer.

    ?proposal_activity prov:wasInformedBy ?proposal_creation_activity.

    OPTIONAL {
    
        ?decision_activity prov:wasInformedBy ?proposal_activity;
            prov:startedAtTime ?decision_time.
    
    }

    FILTER (!BOUND(?decision_time) || ?decision_time >= "2023-01-01"^^xsd:date)


    FILTER (lang(?title) = "de")
    FILTER (lang(?description) = "de")
    FILTER (lang(?answer) = "de")

} ORDER BY ?department ?office

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

## Queries for Parliament-Services

### Proposal

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?title ?proposal_entity ?answer_de ?answer_fr ?answer_it ?decision_time WHERE {
    ?registration_activity a paf:RegistrationActivity.
    ?identifier_entity prov:wasGeneratedBy ?registration_activity;
        schema:identifier ?id.
    ?registration_entity prov:wasGeneratedBy ?registration_activity;
        schema:title ?title. FILTER (lang(?title) = "de")
    ?proposal_creation_activity prov:wasInformedBy+ ?registration_activity.
    ?proposal_entity prov:wasGeneratedBy ?proposal_creation_activity.
    ?proposal_entity chpaf:proposal ?answer_de. FILTER (lang(?answer_de) = "de")
    ?proposal_entity chpaf:proposal ?answer_fr. FILTER (lang(?answer_fr) = "fr")
    ?proposal_entity chpaf:proposal ?answer_it. FILTER (lang(?answer_it) = "it")
    ?proposal_activity prov:wasInformedBy ?proposal_creation_activity.
    OPTIONAL {
        ?decision_activity prov:wasInformedBy ?proposal_activity;
            prov:startedAtTime ?decision_time.
    }
}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

### Information

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?title ?information_entity ?information_de ?information_fr ?information_it WHERE {
    ?registration_activity a paf:RegistrationActivity.
    ?identifier_entity prov:wasGeneratedBy ?registration_activity;
        schema:identifier ?id.
    ?registration_entity prov:wasGeneratedBy ?registration_activity;
        schema:title ?title. FILTER (lang(?title) = "de")
    ?information_creation_activity prov:wasInformedBy+ ?registration_activity.
    ?information_entity prov:wasGeneratedBy ?information_creation_activity.
    ?information_entity chpaf:mopoInformation ?information_de. FILTER (lang(?information_de) = "de")
    ?information_entity chpaf:mopoInformation ?information_fr. FILTER (lang(?information_fr) = "fr")
    ?information_entity chpaf:mopoInformation ?information_it. FILTER (lang(?information_it) = "it")
}
"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

## MoPo with Changed Registration

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?new ?old WHERE {

    ?new prov:wasInformedBy+ ?old;
        a paf:RegistrationActivity.
    
    ?old a paf:RegistrationActivity.

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

or faster if `paf:wasReplacedBy` predicate is set:

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?new ?old WHERE {

    ?old prov:wasGeneratedBy/a paf:RegistrationActivity.

    ?old paf:wasReplacedBy ?new.

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

## All Activities for a Certain Curia Number

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?activity WHERE {

    BIND ("02.3532" as ?id)

    ?identifier_entity schema:identifier ?id;
        prov:wasGeneratedBy ?registration_activity.

    ?activity prov:wasInformedBy* ?registration_activity;
        

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

## Proposal after Information

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?decision_activity prov:wasInformedBy+ ?activity;
        a paf:DecisionActivity.
        
    FILTER EXISTS {
            ?activity a paf:InformationActivity.
    }

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

## Registration Info in Correct Order

With the `values` keyword it can also be selected, which predicates to show.

In [ ]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?p ?o WHERE {

    BIND ("02.3532" as ?id)

    ?identifier_entity schema:identifier ?id;
        prov:wasGeneratedBy ?registration_activity.
    ?registration_entity prov:wasGeneratedBy ?registration_activity;

        ?p ?o.
        
    values (?p ?order) { (schema:identifier 1) (schema:title 2) (schema:author 3) (schema:description 4) }
    
} ORDER BY ?order

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df